In [1]:
import codecs 
import glob 
import multiprocessing 
import os 
import re 
import nltk 
import gensim.models.word2vec as w2v
import numpy as np

C:\Users\Administrator\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [19]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
#Загрузка текста
texts = sorted(glob.glob("texts/*.txt"))
texts

['texts\\1.txt',
 'texts\\10.txt',
 'texts\\11.txt',
 'texts\\12.txt',
 'texts\\13.txt',
 'texts\\14.txt',
 'texts\\15.txt',
 'texts\\16.txt',
 'texts\\17.txt',
 'texts\\18.txt',
 'texts\\19.txt',
 'texts\\2.txt',
 'texts\\20.txt',
 'texts\\3.txt',
 'texts\\4.txt',
 'texts\\5.txt',
 'texts\\6.txt',
 'texts\\7.txt',
 'texts\\8.txt',
 'texts\\9.txt']

In [20]:
#Загрузка стоп-слов
with codecs.open('stopwords.txt', 'r', 'utf-8') as file:
    words = file.read()
    raw_words = tokenizer.tokenize(words)
    clean = (re.sub("[^а-яА-Я]", " ", raw_words[0])).lower()
    stopwords = clean.split()
print(stopwords)

['а', 'бы', 'в', 'во', 'вот', 'для', 'до', 'если', 'же', 'за', 'и', 'из', 'или', 'к', 'ко', 'между', 'на', 'над', 'но', 'о', 'об', 'около', 'от', 'по', 'под', 'при', 'про', 'с', 'среди', 'то', 'у', 'чтобы', 'их', 'при', 'таких', 'также', 'как', 'что', 'того', 'не', 'москва', 'москве', 'вам', 'вас', 'вам', 'вас', 'москвы', 'так', 'м', 'его', 'нас', 'все', 'это', 'наша', 'объ']


In [4]:
#Все тексты в одну большую строку
corpus_raw = ""
for text in texts:
    print('Считывается %s' % text)
    with codecs.open(text,'r','utf-8') as file:
        corpus_raw += file.read()
    print('Сборка длиной %d символов' % len(corpus_raw))
    print()

Считывается texts\1.txt
Сборка длиной 3605 символов

Считывается texts\10.txt
Сборка длиной 6031 символов

Считывается texts\11.txt
Сборка длиной 9343 символов

Считывается texts\12.txt
Сборка длиной 12382 символов

Считывается texts\13.txt
Сборка длиной 14669 символов

Считывается texts\14.txt
Сборка длиной 16154 символов

Считывается texts\15.txt
Сборка длиной 18538 символов

Считывается texts\16.txt
Сборка длиной 20989 символов

Считывается texts\17.txt
Сборка длиной 20989 символов

Считывается texts\18.txt
Сборка длиной 21790 символов

Считывается texts\19.txt
Сборка длиной 26436 символов

Считывается texts\2.txt
Сборка длиной 31316 символов

Считывается texts\20.txt
Сборка длиной 80660 символов

Считывается texts\3.txt
Сборка длиной 84013 символов

Считывается texts\4.txt
Сборка длиной 88532 символов

Считывается texts\5.txt
Сборка длиной 92922 символов

Считывается texts\6.txt
Сборка длиной 96928 символов

Считывается texts\7.txt
Сборка длиной 99507 символов

Считывается texts\8.

In [5]:
#Разбиваем строку на предложения. Каждое предложение с новой строки
raw_sentences = tokenizer.tokenize(corpus_raw)
print('Всего %d предложений' % len(raw_sentences))

Всего 736 предложений


In [6]:
def sentence_to_wordlist(raw):
    clean_data = []
    clean = (re.sub("[^а-яА-Яa-zA-Z]"," ", raw)).lower()
    words = clean.split()
    for word in words:
        if np.isin(word, stopwords) == False and len(word)>2:
            clean_data.append(word)
    return(clean_data)

In [7]:
sentences = []
for raw_sentence in raw_sentences:
    if(len(raw_sentence) > 0):
        sentences.append(sentence_to_wordlist(raw_sentence))

In [8]:
print('Пример:')
print('Предложение:', raw_sentences[13])
print('Получаем:', sentence_to_wordlist(raw_sentences[13]))

Пример:
Предложение: Стоимость услуги в таком случае рассчитывается, исходя из состава отходов, их количества и расстояния до ближайшего места захоронения.
Получаем: ['стоимость', 'услуги', 'таком', 'случае', 'рассчитывается', 'исходя', 'состава', 'отходов', 'количества', 'расстояния', 'ближайшего', 'места', 'захоронения']


In [9]:
token_count = sum([len(sentence) for sentence in sentences])
print('Всего %d токенов' % token_count)

Всего 10002 токенов


In [10]:
#Размер вектора
num_features = 300

#минимальное пороговое значение
min_word_count = 3

#количество потоков
num_workers = multiprocessing.cpu_count() #возвращает к-во ядер процессора

#размер окна
context_size = 7

#Downsample setting for frequent words
downsampling = 1e-3 #Not see too many times the most frequent words...

#Seed for the randomnessss, to make results reproducible
seed = 1

In [11]:
word2Vec = w2v.Word2Vec(sg=1,
                           seed=seed,
                           workers=num_workers,
                           size=num_features,
                           min_count=min_word_count,
                           window=context_size,
                           sample=downsampling
)

In [12]:
word2Vec.build_vocab(sentences)

In [13]:
print('Длина словаря Word2Vec: %d' % word2Vec.corpus_count)

Длина словаря Word2Vec: 736


In [14]:
#Тренируем модель
word2Vec.train(sentences=sentences,total_examples=word2Vec.corpus_count,epochs=word2Vec.iter)

26622

In [15]:
#Сохраняемся
word2Vec.save('word2Vec.w2v')

In [22]:
word2Vec.most_similar("мусор")

[('услуг', 0.9998680949211121),
 ('только', 0.9998652338981628),
 ('отходы', 0.9998636841773987),
 ('снег', 0.9998620748519897),
 ('предоставляем', 0.9998615980148315),
 ('отходов', 0.9998572468757629),
 ('техники', 0.9998534917831421),
 ('утилизации', 0.999853253364563),
 ('можно', 0.9998525381088257),
 ('позволяет', 0.9998493194580078)]